In [1]:
import torch
import json
import math
import hiq
import torch.nn as nn
import torch.nn.functional as F

from pathlib import Path

from llama import ModelArgs, Transformer, Tokenizer, LLaMA
from llama.generation import sample_top_p

### load checkpoint

checkpoint = torch.load('../7B/consolidated.00.pth', map_location="cpu")

In [2]:
with open('../7B/params.json', "r") as f:
    params = json.loads(f.read())

### Initialize model

In [3]:
model_args: ModelArgs = ModelArgs(
    max_seq_len=512, max_batch_size=1, **params
)
tokenizer = Tokenizer('../tokenizer.model')
model_args.vocab_size = tokenizer.n_words
torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = Transformer(model_args)
torch.set_default_tensor_type(torch.FloatTensor)
#model.load_state_dict(checkpoint,strict= False)

generator = LLaMA(model, tokenizer)

In [4]:
temperature: float = 0.8
top_p: float = 0.95
max_seq_len=512
max_batch_size=1

### Generate internative input 

In [14]:
h = torch.randn((1,8,4096), dtype = torch.float16)

In [15]:
freqs_r = torch.randn((8,64,2), dtype = torch.float32)

In [16]:
start_pos = 262
mask = None

In [17]:
import thop
from thop import clever_format

In [18]:
macs, _ = thop.profile(model.layers[0], (h.cuda(), start_pos, freqs_r.cuda(), mask))

[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.


In [19]:
macs

1619001344.0